In [16]:
from concepts import Component, Connection, Assembly, CapabilityInput, CapabilityOutput, Composite, SYSTEMType, InputStates, ELSE, ALWAYS, SYSTEM, printOverview
from components import DrivingShaft, AirOutlet, Axle, InputPlug, DrivenShaft
from typing import Union


# uit concepts
modalities
- class Modality()

connectors
- class Connector(NamedObject)


health
- class Health(Modality)

unreliable
- class Unreliable()

connections
- class Connection(Unreliable, Generic[Tout, Tin])

module
- class Module(NamedObject)

capabilities
- class Capability(Module, CapabilityOutput, Unreliable) 

components (relaties tussen input en output)
- class Component(Module, Unreliable)
- class SuperComponent(Component)

Assemblies (container voor meerdere modules)
- class Assembly(Module)

states
- class State(Module)



In [11]:
#uit connections.py

class Axle(Connection): 
    def __init__(self, input: DrivingShaft, output: DrivenShaft):
        super().__init__(input, output)
    
        self.setNormal([([self.input.moving.YES], [self.output.moving.YES]),
                            (ELSE, [self.output.moving.NO])])
        self.addFailure("broken", 0.001, [(ALWAYS, [self.output.moving.NO])])


    def getDescriptor(self):
        return "axle" 

class Ability(Connection):
    def __init__(self, input:CapabilityOutput, output:CapabilityInput):
        super().__init__(input, output)

    def getDescriptor(self):
        return "ability"


In [17]:
# uit components.py

class Motor(Component):
    def __init__(self, parent, name:str):
        super().__init__(parent, name)
        self.signal = InputPlug(self, "power")
        self.shaft = DrivingShaft(self, "shaft")

        self.setNormal([([self.signal.present.YES] , [self.shaft.moving.YES]),
                        (ELSE                      , [self.shaft.moving.NO])])

        self.addFailure("broken", 0.005, [(ALWAYS, [self.shaft.moving.NO])])


class Fan(Component):
    def __init__(self, parent, name:str):
        super().__init__(parent, name)
        self.shaft = DrivenShaft(self, "shaft")
        self.outlet = AirOutlet(self, "air outlet")

        self.setNormal([([self.shaft.moving.YES] , [self.outlet.present.YES]),
                              (ELSE , [self.outlet.present.NO])])


In [19]:
from collections import Counter

def printOverview(assembly: Assembly, verbose:bool=False) -> None:
    modules = list(assembly.getModules(recursive=True))
    if verbose:
        print(len(modules), "MODULES")
        for m in modules: print(m)
    connections = list(assembly.getConnections(recursive=True))
    connectorsUsed = list(connectors for connection in connections for connectors in connection.getConnectors())
    connectors = set(assembly.getConnectors(recursive=True))
    if verbose:
        print(len(connections), "CONNECTIONS")
        for c in connections: print(c)
    if verbose:
        print(len(connectors), "CONNECTORS")
        for c in connectors: print(c)
    if verbose:
        print(len(connectorsUsed), "CONNECTORS USED")
        for c in connectorsUsed: print(c)
    print("CONNECTORS NOT CONNECTED")
    for c in connectors - set(connectorsUsed) : print(c)
    print("INPUTS NOT CONNECTED")
    for input in assembly.openInputs(): print(input)
    print("CONNECTORS CONNECTED MULTIPLE TIMES")
    counter = Counter(connectorsUsed)
    for c, n in counter.items(): 
        if n>1: print(c, n)

fan = Fan(SYSTEM, "Fan")
printOverview(fan)

setting normal for Fan
CONNECTORS NOT CONNECTED
Fan.shaft
Fan.air outlet
INPUTS NOT CONNECTED
Fan.shaft
CONNECTORS CONNECTED MULTIPLE TIMES
